# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import string
import pandas as pd 
from sqlalchemy import create_engine
import re
import pickle



import nltk
from nltk.tokenize import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

In [18]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')

query = "SELECT * From DisasterResponse"
df = pd.read_sql_query(query, engine,)

X = df['message'].values
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

### 2. Write a tokenization function to process your text data

In [4]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def tokenize(text):
    '''
    Description:
        - Normalization to lowercase
        - Remove punctuation characters
        - Tokenization, lemmatization, and stop word removal
    
    Arguments:
        - text as string
    
    Returns:
        - tokens: a list of tockenized text
    '''
    
    #normalize text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

In [5]:
#test of tokenization function

text = "Sentence first. Sentence second. Sentence third"
tokenize(text)

['sentence', 'first', 'sentence', 'second', 'sentence', 'third']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('forest', MultiOutputClassifier( RandomForestClassifier() ))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state =30, test_size =0.33)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def model_performance(model, x_test, y_test):
    y_predicted = model.predict(x_test)
    y_predicted_df = pd.DataFrame(y_predicted, columns = y_test.columns)

    overall_accuracy = (y_predicted == y_test).mean().mean()
    print("Overall accuracy: {0} \n".format(overall_accuracy))

    for column in y_test.columns:
        print('Feature: {}\n'.format(column))
        print('Accuracy: ', accuracy_score(y_test[column], y_predicted_df[column]))
        print(classification_report(y_test[column],y_predicted_df[column]))
        print('------------------------------------------------------\n')

In [14]:
model_performance(pipeline, X_test, Y_test)

Overall accuracy: 0.9439042584661294 

Feature: related

Accuracy:  0.8064590809121426
             precision    recall  f1-score   support

          0       0.62      0.47      0.54      1983
          1       0.85      0.91      0.88      6598
          2       0.50      0.14      0.22        58

avg / total       0.79      0.81      0.80      8639

------------------------------------------------------

Feature: request

Accuracy:  0.8858664197245052
             precision    recall  f1-score   support

          0       0.90      0.98      0.93      7138
          1       0.80      0.46      0.58      1501

avg / total       0.88      0.89      0.87      8639

------------------------------------------------------

Feature: offer

Accuracy:  0.996064359300845
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8606
          1       0.00      0.00      0.00        33

avg / total       0.99      1.00      0.99      8639

----------

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('forest', MultiOutputClassifier( RandomForestClassifier() ))
    ])

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)) 
}

# another possible parameters - skipped to optimize run time
# 'vect__max_features': (5000, 10000),
#' clf__estimator__min_samples_split': [2, 4],
# ...

grid_search = GridSearchCV(pipeline2, param_grid=parameters, verbose=5, n_jobs = 4)
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] vect__ngram_range=(1, 1) ........................................
[CV]  vect__ngram_range=(1, 1), score=0.2468354430379747, total=  36.9s
[CV] vect__ngram_range=(1, 1) ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.0s remaining:    0.0s


[CV]  vect__ngram_range=(1, 1), score=0.23075607252822442, total=  44.9s
[CV] vect__ngram_range=(1, 1) ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s


[CV]  vect__ngram_range=(1, 1), score=0.23742730071843995, total=  41.3s
[CV] vect__ngram_range=(1, 2) ........................................


...

[CV]  vect__ngram_range=(1, 2), score=0.24871707150188163, total= 1.8min
[CV] vect__ngram_range=(1, 2) ........................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.3min remaining:    0.0s


[CV]  vect__ngram_range=(1, 2), score=0.23092712966130688, total= 1.3min
[CV] vect__ngram_range=(1, 2) ........................................
[CV]  vect__ngram_range=(1, 2), score=0.23588778652069792, total= 1.4min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  7.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
model_performance(grid_search, X_test, Y_test)

Overall accuracy: 0.9422644081748144 

Feature: related

Accuracy:  0.791874059497627
             precision    recall  f1-score   support

          0       0.57      0.50      0.53      1983
          1       0.85      0.88      0.87      6598
          2       0.29      0.28      0.28        58

avg / total       0.78      0.79      0.79      8639

------------------------------------------------------

Feature: request

Accuracy:  0.882162287301771
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      7138
          1       0.79      0.44      0.56      1501

avg / total       0.87      0.88      0.87      8639

------------------------------------------------------

Feature: offer

Accuracy:  0.9961801134390554
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8606
          1       0.00      0.00      0.00        33

avg / total       0.99      1.00      0.99      8639

-----------

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 0.9672415788864452
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      8352
          1       0.64      0.03      0.06       287

avg / total       0.96      0.97      0.95      8639

------------------------------------------------------

Feature: death

Accuracy:  0.9623799050816066
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      8272
          1       0.79      0.16      0.26       367

avg / total       0.96      0.96      0.95      8639

------------------------------------------------------

Feature: other_aid

Accuracy:  0.8700081027896748
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      7526
          1       0.46      0.05      0.09      1113

avg / total       0.82      0.87      0.82      8639

------------------------------------------------------

Feature: infrastructure_related

Accuracy:  0.9312420419029981
        

### 8. Export your model as a pickle file

In [17]:
with open("model.pickle", 'wb') as pickle_file:
    pickle.dump(grid_search, pickle_file)

### 9. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.